In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug 16 06:41:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!mkdir output_competition

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
import os
os.getcwd()
os.environ["WANDB_DISABLED"] = "true"

In [5]:
!pip install -q emoji

     |████████████████████████████████| 184 kB 10.9 MB/s 


In [6]:
!pip install -q simpletransformers

     |████████████████████████████████| 221 kB 13.4 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 124 kB 19.8 MB/s 
     |████████████████████████████████| 8.0 MB 20.5 MB/s 
     |████████████████████████████████| 2.6 MB 45.4 MB/s 
     |████████████████████████████████| 1.2 MB 35.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
     |████████████████████████████████| 264 kB 46.1 MB/s 
     |████████████████████████████████| 1.6 MB 36.2 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 636 kB 40.1 MB/s 
     |████████████████████████████████| 170 kB 50.2 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 133 kB 48.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 243 kB 53.6 MB/s 
     |████████████████████████████████| 118 kB 53.3 MB/s 
     |██████████████

In [7]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd

import emoji
import warnings
import re
warnings.filterwarnings('ignore')
from tqdm import tqdm

import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(13)

In [8]:
def give_emoji_free_text(text):
    allchars = [str for str in text.encode().decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.encode().decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

def cleaning_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'\d+', '', text)
  text = text.replace("&amp"," ")
  text = " ".join(text.split())
  return text


In [9]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [10]:
train_df=pd.read_csv("/content/gdrive/MyDrive/Hackathon/Train.csv")
# train_df['tweet']=train_df['tweet'].apply(cleaning_text)
# train_df['tweet']=train_df['tweet'].apply(give_emoji_free_text)


test_df=pd.read_csv("/content/gdrive/MyDrive/Hackathon/Test.csv")
# test_df['tweet']=test_df['tweet'].apply(cleaning_text)
# test_df['tweet']=test_df['tweet'].apply(give_emoji_free_text)


train_df['type']=train_df['type'].map({'sexual_violence':0,'Physical_violence':1,'emotional_violence':2,
                                       'economic_violence':3,'Harmful_Traditional_practice':4})

In [11]:
train_df['target'] = train_df['type'].values
test_df['target'] = 0

train1=train_df.copy()
train1=train1[['tweet','target']]
test1=test_df.copy()
test1=test1[['tweet','target']]
train1.shape,test1.shape

((39650, 2), (15581, 2))

In [12]:
y_pred_tot=[]
# err=[]

fold=KFold(n_splits=5, shuffle=True, random_state=42)
i=1
for train_index, test_index in tqdm(fold.split(train1,train1['target'])):
    train1_trn, train1_val = train1.iloc[train_index], train1.iloc[test_index]
    model = ClassificationModel('bert', '/content/gdrive/MyDrive/Hackathon/zindi_bert_base',num_labels=5, use_cuda=True, args={'train_batch_size':32,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': True,
                                                                         'do_lower_case': True,
                                                                         'num_train_epochs': 5,
                                                                         'max_seq_length': 80,
                                                                         'regression': False,
                                                                         'manual_seed': 13,
                                                                         "learning_rate":3e-5,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "early_stopping_consider_epochs":True,
                                                                         "early_stopping_patience":2,
                                                                         "silent": True})
    model.train_model(train1_trn)
    
    raw_outputs_val = model.eval_model(train1_val)[1]
    
    raw_outputs_val = softmax(raw_outputs_val,axis=1)
    
    predictions=np.argmax(raw_outputs_val,axis=1)
    
    print(f"Accuracy: {accuracy_score(train1_val['target'], predictions)}")
    
    raw_outputs_test = model.eval_model(test1)[1]
    
    raw_outputs_test = softmax(raw_outputs_test,axis=1)
    
    
    y_pred_tot.append(raw_outputs_test)

0it [00:00, ?it/s]Some weights of the model checkpoint at /content/gdrive/MyDrive/Hackathon/zindi_bert_base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from t

Accuracy: 0.9994955863808322


1it [24:48, 1488.58s/it]Some weights of the model checkpoint at /content/gdrive/MyDrive/Hackathon/zindi_bert_base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized 

Accuracy: 0.9998738965952081


2it [49:25, 1481.66s/it]Some weights of the model checkpoint at /content/gdrive/MyDrive/Hackathon/zindi_bert_base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized 

...

3it [1:14:07, 1481.77s/it]Some weights of the model checkpoint at /content/gdrive/MyDrive/Hackathon/zindi_bert_base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Accuracy: 0.9993694829760403


4it [1:38:42, 1478.98s/it]Some weights of the model checkpoint at /content/gdrive/MyDrive/Hackathon/zindi_bert_base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Accuracy: 0.9991172761664565


5it [2:03:14, 1478.83s/it]


In [13]:
ss=pd.read_csv('/content/gdrive/MyDrive/Hackathon/Sample_submission.csv')
ss.head()

,Tweet_ID,type
0,ID_D9ONL553,NaN
1,ID_263YTILY,NaN
2,ID_62VS6IXC,NaN
3,ID_OG50429Q,NaN
4,ID_VNQO3TJG,NaN


In [14]:
probs            = np.mean(np.array(y_pred_tot),axis=0)
probdf           = pd.DataFrame(probs,columns = ['sexual_violence','Physical_violence','emotional_violence','economic_violence','Harmful_Traditional_practice'])
probdf['tweet'] = test_df['tweet']
probdf['Tweet_ID'] = test_df['Tweet_ID']

probdf[['Tweet_ID','tweet']+['sexual_violence','Physical_violence','emotional_violence','economic_violence','Harmful_Traditional_practice']].to_csv('./output_competition/probability_output.csv',index=False)


In [15]:
ss['type']=np.argmax(np.mean(np.array(y_pred_tot),axis=0),axis=1)
ss['type'].value_counts()

0    11260
4     3100
2      659
3      498
1       64
Name: type, dtype: int64

In [16]:
ss['type'].value_counts(normalize=True)

0    0.722675
4    0.198960
2    0.042295
3    0.031962
1    0.004108
Name: type, dtype: float64

In [17]:
ss['type']=ss['type'].map({0:'sexual_violence',1:'Physical_violence',2:'emotional_violence',
                     3:'economic_violence',4:'Harmful_Traditional_practice'})

ss['type'].value_counts(normalize=True)

sexual_violence                 0.722675
Harmful_Traditional_practice    0.198960
emotional_violence              0.042295
economic_violence               0.031962
Physical_violence               0.004108
Name: type, dtype: float64

In [18]:
ss.to_csv('bert_base_best_result.csv',index=False)